Planning this script. Steps:
* 0. Set parameters (will be part of pipeline so this block will be silenced)
  1. Setup:
        * Paths
        * Utils functions
        * Load and check config file
  2. Load Data
        * **Routine data** (DHIS2) already formatted & aggregated (output of pipeline XXX)
        * **Population data** (DHIS2) already formatted & aggregated (output of pipeline YYY) & aggregated at **ADM2 x YEAR** level (⚠️ how to implement this better!⚠️)
        * (optional) **Care seeking (taux recherche soins)** (DHS)
        * (user choice) **Reporting Rate**, pick one of:
            * "**SNIS**": pre-cumputed (directly downloadable from SNIS DHIS2 instance) and formatted&aligned elsewhere (output of pipelibe ZZZ ⚠️ YET TO DO ⚠️)
            * "**ANY**
            * "**CONF** / "Fre" way (based on code in nb: `~/dhis2_incidence/code/WIP/code_from_fre/DRC_DHIS2_analyses_fvdb_v2.ipynb`)
  3. Calculate **Incidence**
        * define **functions** - ⚠️ functions not needed if we only run it once (1 RR only, user choice) -> **make script** so it's easer to debug ⚠️
            * `caluclate_monthly_cases()`
            * `calculate_yearly_incidence()`: Crude, Adjusted 1 (Testing), Adjusted 2 (Reporting), (optional) Adjusted 3 (Care Seeking)
  7. **Export** Incidence data to **Datasets**
  8. 🚧 Reporting: data inspection (plots and summary tables) - this might go to dedicated nb ...

-------------------
**Naming harmonization to improve code readability:** 🚧 IN PROGRESS 🚧

**Incidence**, COLUMN NAMES (⚠️ to be capitalized!):
* "crude_incidence" = "Crude"
* "incidence_adj_testing" = "Adjusted 1 (Testing)"
* "incidence_adj_reporting" = "Adjusted 2 (Reporting)"
* _"incidence_adj_careseeking" = "Adjusted 3 (Careseeking)"_ ⚠️is this good naming?

**Reporting Rate** data frames, based on different **methods**:
* follwo this structure: reporting\_rate\_\<method\>\_\<periodicity\>. So:
    * **SNIS**: `reporting_rate_snis_year` , `reporting_rate_snis_month`
    * **CONF** (Diallo 2025): `reporting_rate_conf_year` , `reporting_rate_conf_month`
    * **ANY** (Ousmane's, BFA): `reporting_rate_any_year` , `reporting_rate_any_month` 

--------------------

## IMPORTANT:
* "**SNT_config.json**" **changes country often** (depends on when Esteban works on it) so you might need to manually set it to "SNT_config_**BFA**.json" or "SNT_config_**DRC**.json" if the code cannot find the routine data file ... !
* **Population** data is **pre-formatted** in "pipelines/dhis2_incidence/code/WIP/**TEMP_population_formatting**.ipynb"! Might need to **re-run that nb with a different config file**
    * Definitely re-run "TEMP_population_formatting.ipynb" if you get `ADM2_POP` and all incidences as `NA` (section 4) - because that means that it's using the wrong POP data (e.g., BFA instead of DRC or vice versa, which happens when config file is modified by Esteban)
* Currently using **mock data** for 1) **Careseeking** and 2) **RR from SNIS** to test code <-- remove/replace with actual tables!

## TO DO:
* <s>define and implememnt logic to handle both presence and absenced of **care seeking** data! At the moment I don't calculate N3 (but I have the code for it in DRC workspace)</s>
* <s>save output tables in **Datasets**</s>
* <s>add logic to conditional **import** of **CARESEEKING data** so that if not there (prints message/warning and) it just skips without breaking code ... </s>
* <s>**Move Reporting Rate** calculations and analysis (plots and summary stats) to a separate notebook and pipeline!!</s>
* <s>**Update reporting rate data import**, to be from `/data/reporting_rate/`</s>
* <s>Once _Reporting Rate_ is saved to _Datasets_ (still WIP), modify code at chunk `# Load reporting rate data based on choice` (see # comments in there, basically comment/uncomment existing code as "switch")</s>
* <s>Make this nb into a **pipeline**</s>
* **update path for `FORMATTED_POP_DATA_PATH`** <- Once DHIS2 formatting pipelines gets POP data already aggregated to YEAR * ADM2 (read from **DATASETs** instead!)

## 0. Parameters
👇 these are now ⚡**pipeline parameters**⚡!

In [ ]:
# Parameters
# ROOT_PATH <- "~/workspace" 
# CARESEEKING_DATA_FILENAME <- "~/workspace/data/dummy_data/COD_mock_trs.csv"
# REPORTING_RATE_CHOICE <- "conf"

## 1. Setup

### 1.1. Paths

In [ ]:
# PROJECT PATHS
CODE_PATH <- file.path(ROOT_PATH, 'code') # this is where we store snt_functions.r and snt_utils.r
CONFIG_PATH <- file.path(ROOT_PATH, 'configuration') # .json config file
DATA_PATH <- file.path(ROOT_PATH, 'data') # same as in Datasets but /data/ gets over written every time a new version of Datasets is pushed

# ⚠️ TEMPORARY - until DHIS2 formatting pipelines get POP data already aggregated to YEAR * ADM2 ⚠️
#    ... then read from DATASETs instead!
FORMATTED_POP_DATA_PATH <- '/home/hexa/workspace/pipelines/dhis2_incidence/code/WIP'

# TEMPORARY solution to test code: use MOCK data (see `00_TEMP_make_mock_TRS.ipynb`) 
MOCK_DATA_PATH <- file.path(DATA_PATH, "dummy_data" ) # data/dummy_data/COD_mock_rr_snis.rds

### 1.2. Utils functions

In [ ]:
source(file.path(CODE_PATH, "snt_utils.r"))

### 1.3. Packages

In [ ]:
# List required pcks  ---------------->  check  what are the really required libraries
required_packages <- c("arrow", # for .parquet
                       "tidyverse",
                       "stringi", 
                       "sf",
                       "jsonlite", 
                       "httr", 
                       "reticulate")

# Execute function
install_and_load(required_packages)

#### For 📦{sf}, tell OH where to find stuff ...

In [ ]:
# Hope this gets fixed at the source one day ...
Sys.setenv(PROJ_LIB = "/opt/conda/share/proj")
Sys.setenv(GDAL_DATA = "/opt/conda/share/gdal")

#### Set environment to load openhexa.sdk from the right path

In [ ]:
# Set environment to load openhexa.sdk from the right path
Sys.setenv(RETICULATE_PYTHON = "/opt/conda/bin/python")
reticulate::py_config()$python
openhexa <- import("openhexa.sdk")

### 1.4. Load and check `config` file

In [ ]:
# Load SNT config
CONFIG_FILE_NAME <- "SNT_config.json"
config_json <- tryCatch({ fromJSON(file.path(CONFIG_PATH, CONFIG_FILE_NAME)) },
    error = function(e) {
        msg <- paste0("Error while loading configuration", conditionMessage(e))  
        cat(msg)   
        stop(msg) 
    })

msg <- paste0("SNT configuration loaded from  : ", file.path(CONFIG_PATH, CONFIG_FILE_NAME)) 
log_msg(msg)

**Checks for SNT mandatory configuration fields**

In [ ]:
# CHECK SNT configuration 
snt_config_mandatory <- c("COUNTRY_CODE", "DHIS2_ADMINISTRATION_1", "DHIS2_ADMINISTRATION_2") 

for (conf in snt_config_mandatory) {
    print(paste(conf, ":", config_json$SNT_CONFIG[conf]))
    if (is.null(config_json$SNT_CONFIG[[conf]])) {
        msg <- paste("Missing configuration input:", conf)
        cat(msg)   
        stop(msg)
    }
}

**Save config fields as variables**

In [ ]:
# Generic
COUNTRY_CODE <- config_json$SNT_CONFIG$COUNTRY_CODE
ADMIN_1 <- toupper(config_json$SNT_CONFIG$DHIS2_ADMINISTRATION_1)
ADMIN_2 <- toupper(config_json$SNT_CONFIG$DHIS2_ADMINISTRATION_2)

# Specific to INCIDENCE calculation (this nb)
# How to treat 0 values (in this case: "SET_0_TO_NA" converts 0 to NAs)
NA_TREATMENT <- config_json$SNT_CONFIG$NA_TREATMENT

# Which (aggregated) indicators to use to evaluate "activity" of an HF - for Reporting Rate method "ANY"
DHIS2_INDICATORS <- names(config_json$DHIS2_DATA_DEFINITIONS$DHIS2_INDICATOR_DEFINITIONS)

In [ ]:
# Fixed routine formatting columns
fixed_cols <- c('OU','PERIOD', 'YEAR', 'MONTH', 'ADM1_ID', 'ADM1', 'ADM2_ID', 'ADM2') # use `OU` as it contains unique ids (OU_NAME has homonimous values!)
print(paste("Fixed routine data (\"dhis2_routine\") columns (always expected): ", paste(fixed_cols, collapse=", ")))

## 2. Load Data

### 2.1. **Routine** data (DHIS2) 
already formatted & aggregated (output of pipeline XXX)

In [ ]:
# DHIS2 Dataset extract identifier
dataset_name <- config_json$SNT_DATASET_IDENTIFIERS$DHIS2_DATASET_FORMATTED

# Load file from dataset
dhis2_routine <- tryCatch({ get_latest_dataset_file_in_memory(dataset_name, paste0(COUNTRY_CODE, "_routine.parquet")) }, 
                  error = function(e) {
                      msg <- paste("Error while loading DHIS2 routine data file for: " , COUNTRY_CODE, conditionMessage(e))  # log error message
                      cat(msg)
                      stop(msg)
})

msg <- paste0("DHIS2 routine data loaded from dataset : ", dataset_name, " dataframe dimensions: ", paste(dim(dhis2_routine), collapse=", "))
log_msg(msg)

In [ ]:
head(dhis2_routine)

### 2.2. **Population** data (DHIS2) at **ADM2** x **YEAR** level 
already formatted & aggregated (output of pipeline YYY) and then further aggregated in `pipelines/dhis2_incidence/code/WIP/00_TEMP_population_formatting.ipynb`

Note: must run **00_TEMP_population_formatting**.ipynb first!

### ⚠️This to be replaced with code that imports from Datasets ⚠️
🚨 **Expected table** with these cols (bold = **must have**): ADM1, ADM1_ID, **ADM2**, ADM2_ID, **YEAR**, **ADM2_POP**

In [ ]:
# DHIS2 Dataset extract identifier
dhis2_formatted_dataset <- config_json$SNT_DATASET_IDENTIFIERS$DHIS2_DATASET_FORMATTING

# Load file from dataset
dhis2_population_adm2 <- tryCatch({ get_latest_dataset_file_in_memory(dhis2_formatted_dataset, paste0(COUNTRY_CODE, "_population.parquet")) }, 
                  error = function(e) {
                      msg <- paste("Error while loading DHIS2 population file for: " , COUNTRY_CODE, conditionMessage(e))  # log error message
                      cat(msg)
                      stop(msg)
})

msg <- paste0("DHIS2 population data loaded from dataset : ", dhis2_formatted_dataset, " dataframe dimensions: ", paste(dim(dhis2_population_adm2), collapse=", "))
log_msg(msg)

In [ ]:
dhis2_population_adm2 <- read_csv(file.path(FORMATTED_POP_DATA_PATH, paste0(COUNTRY_CODE, "_dhis2_population_adm2.csv")),
                                 show_col_types = FALSE)

dim(dhis2_population_adm2)
head(dhis2_population_adm2)

### 2.3. ❌ (optional) **Care seeking (taux recherche soins)** (DHS)
Currently missing ❌ so using **mock data** (made here: pipelines/dhis2_incidence/code/WIP/**00_TEMP_make_mock_TRS**.ipynb)

💡 Also, TBD: **how to be flexible regarding the input**, as this could come from **DHS, MAP or other sources** ... <br>
=> to do: make **dedicated pipeline for formatting, alignement and checks**

In [ ]:
if (!is.null(CARESEEKING_DATA_FILENAME)) {
    # careseeking_data <- readRDS(CARESEEKING_DATA_FILENAME)
    careseeking_data <- read.csv(CARESEEKING_DATA_FILENAME, header = TRUE, sep = ",", stringsAsFactors = FALSE)
    print("Successfully imported care-seeking data")
    print(dim(careseeking_data))
    head(careseeking_data)
}

### 2.4. **Reporting Rate** (parametrized choice)  
Import Reporting Rate file based on user choice (parameter)

🚨 **Important**: reporting rate must be **monthly**!

In [ ]:
# REPORTING_RATE_CHOICE = "dummy-snis"
# REPORTING_RATE_CHOICE = "any"
# REPORTING_RATE_CHOICE = "conf"

In [ ]:
# Load reporting rate data based on choice

if (REPORTING_RATE_CHOICE == "dummy-snis") {
    rr_dummysnis_file_path <- file.path(MOCK_DATA_PATH, paste0(COUNTRY_CODE, "_mock_rr_snis.rds"))
    reporting_rate_month <- readRDS(rr_dummysnis_file_path)

    # ----------------------------------------------------------------------------------------
    
    # ⚠️ Modify this to TEMPORARILY read from /data/reporting_rate/ INSTEAD of reading from Datasets ! ⚠️
    #  ⚠️ DO NOT DELETE!! ⚠️

    
} else if (REPORTING_RATE_CHOICE %in% c("conf", "any")) {
    rr_dataset_name <- config_json$SNT_DATASET_IDENTIFIERS$DHIS2_DATASET_REPORTING_RATE    
    rr_file_name <- paste0(COUNTRY_CODE, "_reporting_rate_", REPORTING_RATE_CHOICE, "_month.parquet")    
    reporting_rate_month <- tryCatch({get_latest_dataset_file_in_memory(rr_dataset_name, rr_file_name)}, 
                                     error = function(e) {
                                         msg <- paste("Error while loading file", rr_file_name, ":", conditionMessage(e))
                                         cat(msg, "\n")
                                         stop(msg)
                                         })
} else {
  stop("Invalid REPORTING_RATE_CHOICE. Must be 'dummy-snis', 'conf', or 'any'.")
}

# # ⚠️ TEMPORARY code to read from /data/ NOT from Datasets (comment this out and uncomment code above) ⚠️
# } else if (REPORTING_RATE_CHOICE == "conf") {
#     rr_method = REPORTING_RATE_CHOICE
#   rr_file_path <- file.path(DATA_PATH, "dhis2_reporting_rate", paste0(COUNTRY_CODE, "_reporting_rate_", rr_method,"_month.csv")) # data/reporting_rate/COD_reporting_rate_any_month.csv
#   reporting_rate_month <- read_csv(rr_file_path,
#                              show_col_types = FALSE)
#     } else if (REPORTING_RATE_CHOICE == "any") {
#     rr_method = REPORTING_RATE_CHOICE
#   rr_file_path <- file.path(DATA_PATH, "dhis2_reporting_rate", paste0(COUNTRY_CODE, "_reporting_rate_", rr_method,"_month.csv"))
#   reporting_rate_month <- read_csv(rr_file_path,
#                              show_col_types = FALSE)
#     } else {
#   stop("Invalid REPORTING_RATE_CHOICE. Must be 'dummy-snis', 'conf', or 'any'.")
# }
 
# ----------------------------------------------------------------------------------------

# Validate the reporting_rate dataframe
required_cols <- c("ADM2", "YEAR", "MONTH", "REPORTING_RATE")
missing_cols <- setdiff(required_cols, names(reporting_rate_month))
if (length(missing_cols) > 0) {
  stop(paste("Missing required columns:", paste(missing_cols, collapse = ", ")))
}

# Check column types and values
validation_errors <- list()

# Check ADM2 is character
if (!is.character(reporting_rate_month$ADM2)) {
  validation_errors <- c(validation_errors, "ADM2 must be a character vector")
}

# Check YEAR is integer or can be converted, and is recent (current year - 5)
current_year <- as.integer(format(Sys.Date(), "%Y"))
min_valid_year <- current_year - 5

reporting_rate_month <- reporting_rate_month %>%
  mutate(YEAR = suppressWarnings(as.integer(YEAR)))

if (any(is.na(reporting_rate_month$YEAR))) {
  validation_errors <- c(validation_errors, "YEAR contains non-integer values")
} else if (any(reporting_rate_month$YEAR < min_valid_year, na.rm = TRUE)) {
  validation_errors <- c(validation_errors, 
                        paste("Some YEAR values are too old (minimum valid year is", min_valid_year, 
                              "). Found years:", 
                              paste(unique(reporting_rate_month$YEAR[reporting_rate_month$YEAR < min_valid_year]), collapse = ", ")))
}

# Check MONTH is integer or can be converted to, and ranges betweeen 1 and 12
#  ⚠️ TO ADD!

# Check REPORTING_RATE is numeric between 0 and 1
if (!is.numeric(reporting_rate_month$REPORTING_RATE)) {
  validation_errors <- c(validation_errors, "REPORTING_RATE must be numeric")
} else if (any(reporting_rate_month$REPORTING_RATE < 0 | reporting_rate_month$REPORTING_RATE > 1, na.rm = TRUE)) {
  validation_errors <- c(validation_errors, "REPORTING_RATE must be between 0 and 1 (inclusive)")
}

# Report any validation errors
if (length(validation_errors) > 0) {
  stop(paste("Data validation failed:\n", paste("-", validation_errors, collapse = "\n")))
}

# If all checks passed, show the data
cat("Reporting rate data from method '", REPORTING_RATE_CHOICE,"' loaded successfully:\n")
head(reporting_rate_month)

### 2.5. **Shapes** for plotting maps (choropleths)

In [ ]:
# DHIS2 Dataset extract identifier
dataset_name <- config_json$SNT_DATASET_IDENTIFIERS$DHIS2_DATASET_FORMATTED

# Load file from dataset
dhis2_shapes <- tryCatch({ get_latest_dataset_file_in_memory(dataset_name, paste0(COUNTRY_CODE, "_shapes.geojson")) }, 
                  error = function(e) {
                      msg <- paste("Error while loading DHIS2 shapes data file for: " , COUNTRY_CODE, conditionMessage(e))  # log error message
                      cat(msg)
                      stop(msg)
})

msg <- paste0("DHIS2 shapes data loaded from dataset : ", dataset_name, " dataframe dimensions: ", paste(dim(dhis2_shapes), collapse=", "))
log_msg(msg)

In [ ]:
names(dhis2_shapes)

-------------------------------

## 3. Calculate Incidence

Use **functions** to calculate **monthly cases** (`calculate_monthly_cases()`) and then use the output to calculate **yearly incidence** (`calculate_yearly_incidence()`).

This offers the advantage that incidences can be easily recalculated using different input data for reporting_rate (from different methods). 

💡💡💡 <br>
However, since in **now we only import 1 reporting rate** (defined as user choice), **consider taking code out of function** to imrpove readability <br>
(code should only be wrapped in function if need to run multoiple times ... else is just a pain to debug ... !) <br>
💡💡💡

### 3.0. Define Functions

#### **Calculate monthly case data:** `calculate_monthly_cases()`
These methods follow the standard WHO approach for estimating malaria incidence from routine health information systems (WHO, 2023).
As shown in the code, we begin by calculating **monthly malaria case metrics** (confirmed, tested, presumed) at the **ADM2** level and join them with the **monthly reporting rate**. 

This allows us to compute the **test positivity rate** (TPR, where `TPR` = `CONF` / `TEST`) and adjust for incomplete testing using the formula: 
> N1 = C + (P × C/T)

Which is equivalent to:
> N1 = C + (P × TPR)

where:
- N1 = cases adjusted for testing gaps 
- C = confirmed cases (`CONF`)
- P = presumed cases (either `SUSP` - `TEST` or directly available as `PRES`)
- T = tested cases (`TEST`)
- TPR = test positivity rate (`CONF` / `TEST`)
  
This produces `N1`, the number of cases adjusted for testing gaps, calculated at the monthly level in line with WHO recommendations to capture intra-annual variation.

Next, we adjust for incomplete reporting using: 
> N2 = N1 / R

where **R** is the monthly **reporting rate** (i.e., reports received divided by reports expected).

Finally, _if_ **careseeking** data is **available**, N3 is calculated as follows:
> N3 = N2 + (N2 * PROP_PRIV / PROP_PUBL) + (N2 * NO_TREAT / PROP_PUBL)

where:
- PROP_PRIV = proportion of "enfants" (***can we say <5 yo?***) treated in the **private** sector
- PROP_PUBL = proportion of "enfants" (***can we say <5 yo?***) treated in the **public** sector
- NO_TREAT = proportion of "enfants" (***can we say <5 yo?***) which **did not receive any treatment**

Note that this assumes the same TPR across all sectors (private and public).

**Important note** In case reporting rate equals zero (none of the health facilities reported in a given month), N2 is set to `NA`. Note that the annual N2 will be underestimated, which is preferable compared to having Inf values.

The function expects (input):
* **routine_data**: DHIS2 routine data, formatted and aggregated at ADM2 and MONTH level. Tibble (df) _must_ contain the following cols: `YEAR`, `MONTH`, `ADM2`, `CONF`, `TEST`, `SUSP`, `PRES`.  
* **reporting_rate_data**: reporting rate calculated at ADM2 and MONTH level and expressed as proprtion **(0-1)**. Tibble (df) _must_ contain the following cols: `ADM2`, `YEAR`, `MONTH`, `reporting_rate`
* **use_imputed**: ... to be discussed ... default `FALSE`
* **verbose**: ...

The funcion produces (output):
* data frame with the following cols: `ADM2`, `YEAR`, `MONTH`, "value_" * (`CONF`, `TEST`, `SUSP`, `PRES`), `TPR`, `N1`, `N2`

In [ ]:
# GP: temporarily silenced 🤫 `use_imputed` part to be able to run with current data

calculate_monthly_cases <- function(routine_data, reporting_rate_data, careseeking_data = NULL, use_imputed = FALSE, verbose = TRUE) {
    # suffix <- if (use_imputed) "value_imputed_" else "value_"
    
    # Ensure correct data type for numerical columns
    routine_data <- routine_data %>%
    mutate(across(c(YEAR, MONTH, CONF, TEST, SUSP, PRES), as.numeric))
  
    reporting_rate_data <- reporting_rate_data %>%
    mutate(across(c(YEAR, MONTH, REPORTING_RATE), as.numeric))

    summary_data <- routine_data %>%
        # group_by(ADM2, YEAR, MONTH) %>%
        group_by(ADM1, ADM2, YEAR, MONTH) %>% # ADM1 needed to join careseeking data
        summarise(
          # CONF = sum(.data[[paste0(suffix, "CONF")]], na.rm = TRUE),
          # TEST = sum(.data[[paste0(suffix, "TEST")]], na.rm = TRUE),
          # SUSP = sum(.data[[paste0(suffix, "SUSP")]], na.rm = TRUE),
          # PRES = sum(.data[[paste0(suffix, "PRES")]], na.rm = TRUE),
          CONF = sum(CONF, na.rm = TRUE),
          TEST = sum(TEST, na.rm = TRUE),
          SUSP = sum(SUSP, na.rm = TRUE),
          PRES = sum(PRES, na.rm = TRUE),
          .groups = "drop"
        ) %>%
        # type_convert() %>% # *** # automatically detect and convert columns to their proper types
        left_join(reporting_rate_data, 
                  # reporting_rate_data %>% type_convert(), # ***
                  by = c("ADM2", "YEAR", "MONTH")) %>%
        mutate(
          TPR = CONF / TEST,
          N1 = CONF + ((SUSP-TEST) * TPR), # = CASES_ADJUSTED_PRESUMED
          #N1 = CONF + (PRES * TPR),
          N2 = ifelse(REPORTING_RATE == 0, NA_real_, N1 / REPORTING_RATE) # ifelse(reporting_rate == 0, NA_real_, N1 / reporting_rate)  # = CASES_ADJUSTED_PRESUMED_RR      
        )

    if (!is.null(careseeking_data)) {  
      # Added calculations of N3 incidence adjusted for careseeking behaviour 
        summary_data <- summary_data %>%
        mutate(YEAR = as.numeric(YEAR)) %>% # Still needed as safety
        left_join(., careseeking_data, 
                  careseeking_data %>% type_convert(), # ***
                  by = c("ADM1", "YEAR")) %>% # by = join_by(ADM1)
        mutate(
            # add all pieces that are needed to calculate 3rd level of adjustemnt 
         # ADJ_F = (cases_adjusted_presumed_RR/PROP_PUBL)*PROP_PRIV, # Ousmane
          # ADJ_F = (N2 / PROP_PUBL) * PROP_PRIV,  # GP
         # ADJ_T = (cases_adjusted_presumed_RR/PROP_PUBL)*NO_TREAT,  # Ousmane
          # ADJ_T = (N2 / PROP_PUBL) * NO_TREAT, # GP
         # cases_adjusted_presumed_RR_TSR = cases_adjusted_presumed_RR + ADJ_F + ADJ_T, # Ousmane
            N3 = N2 + (N2 * PROP_PRIV / PROP_PUBL) + (N2 * NO_TREAT / PROP_PUBL) # Formula from Rapport Stratification Burkina (same as lines above but easier to read imo)
        )
    }
    
    if (verbose) {
        zero_reporting <- summary_data %>%
                filter(REPORTING_RATE == 0) %>%
                summarise(
                    n_months_zero_reporting = n(),
                    affected_zones = n_distinct(ADM2)
                )
        
        cat("Note:", 
            zero_reporting$n_months_zero_reporting, 
            "ADM2-months had zero reporting rate across",
            zero_reporting$affected_zones, 
            "zones. N2 values were set to NA\n")
    }
    
    return(summary_data)    
}

#### **Compute yearly incidence:** `calculate_yearly_incidence()`
After calculating N1 and N2 for each `ADM2`-`MONTH`, we aggregate the data annually to compute the yearly totals (sums) for crude cases (`CONF`), `N1` and `N2`. Finally, we compute:
* Crude incidence: C / POP × 1000
* Incidence adjusted for testing: N1 / POP × 1000
* Incidence adjusted for testing and reporting: N2 / POP × 1000
* Incidence adjusted for testing, reporting and careseeking behaviour (optional): N3 / POP × 1000

The function expects (input):
* **case_data**: as the output of `calculate_monthly_cases()`, or a tibble/data frame with the following cols: `ADM2`, `YEAR`, `MONTH`, "value_" * (CONF, TEST, SUSP, PRES), `TPR`, `N1`, `N2`  
* **population_data**: df of population data formatted and aligned, aggregated at ADM2 and YEAR level. A tibble/data frame that _must_ contain the following cols: `ADM2`, `YEAR`, `ADM2_POP`

The function produces (output): ⚠️ **TO BE CONFIRMED! ⚠️ Also capitalize all! ⚠️ Also add logic for CARE SEEKING - ADJ N3**
* a data frame with the following cols: ADM2, YEAR, CONF, N1, N2, `crude_incidence`, `incidence_adj_testing`, `incidence_adj_reporting`

In [ ]:
calculate_yearly_incidence <- function(case_data, population_data) {
  # ---- 1. Enforce column types upfront ----
  case_data <- case_data %>%
    mutate(across(where(is.numeric), as.numeric))  # Convert all numeric columns
  
  population_data <- population_data %>%
    mutate(across(c(YEAR, ADM2_POP), as.numeric))
  
  # ---- 2. Core calculation ----
  yearly_incidence <- case_data %>%
    group_by(ADM2, YEAR) %>%
    summarise(
      across(c(CONF, N1, N2), ~sum(., na.rm = TRUE)),
      .groups = "drop"
    ) %>%
    left_join(
      population_data,
      by = c("ADM2", "YEAR")
    ) %>%
    mutate(
      crude_incidence = CONF / ADM2_POP * 1000,
      incidence_adj_testing = N1 / ADM2_POP * 1000,
      incidence_adj_reporting = N2 / ADM2_POP * 1000
    )
  
  # ---- 3. Optional careseeking adjustment ----
  if (CARESEEKING_DATA_AVAILABLE && "N3" %in% names(case_data)) {
    n3_data <- case_data %>%
      group_by(ADM2, YEAR) %>%
      summarise(N3 = sum(N3, na.rm = TRUE),
                .groups = "drop")
    
    yearly_incidence <- yearly_incidence %>%
      left_join(n3_data, by = c("ADM2", "YEAR")) %>%
      mutate(
        incidence_adj_careseeking = N3 / ADM2_POP * 1000
      )
  }
  
  return(yearly_incidence)
}

### 3.1. Use Functions to calculate monthly cases and then incidence

Calculate 1) monthly cases -> 2) yearly incidence

In [ ]:
monthly_cases <- calculate_monthly_cases(
    routine_data = dhis2_routine,
    reporting_rate_data = reporting_rate_month, 
    careseeking_data = careseeking_data
    use_imputed = FALSE,
    verbose = TRUE
)

head(monthly_cases, 3)

In [ ]:
yearly_incidence <- calculate_yearly_incidence(
    case_data = monthly_cases, 
    population_data = dhis2_population_adm2
)

head(yearly_incidence, 3)

## 4. Export

### 4.1. To `/data/dhis2_incidence/` folder
Dynamically include **reporting rate method** used (`rr-method-`) in **filename**

#### .CSV

In [ ]:
# Dynamically include reporting rate method used (`rr-method-`) in filename 
write_csv(yearly_incidence,
          file.path(DATA_PATH, "dhis2_incidence", paste0(COUNTRY_CODE, "_incidence_year_rr-method-", REPORTING_RATE_CHOICE, ".csv"))      
         )

#### .parquet

In [ ]:
arrow::write_parquet(yearly_incidence,
                     file.path(DATA_PATH, "dhis2_incidence", paste0(COUNTRY_CODE, "_incidence_year_rr-method-", REPORTING_RATE_CHOICE, ".parquet"))
                    )

## 4.2. 🗃️ To workspace **Dataset**: "**reporting-rate-computed**" 
Use python code in R kernel

In [ ]:
# Double check that {reticulate} finds the module ... 
# openhexa

### 4.2.0. Create new **dataset** (coded)
This step could be skipped if preference for creating the Dataset manually  ...

Check if a dataset with slug == "**dhis2-incidence**" already exists, if not create one.<br>
This avoids silentily creating a new dataset with same name (yet different slug and id) at each run ...

In [ ]:
# # Define Dataset 
# DATASET_SLUG = "dhis2-incidence"
# DATASET_NAME <- "DHIS2_INCIDENCE"
# DATASET_DESCRIPTION <- "Incidence calculated from DHIS2 routine data. Adjustement for Reporting Rate uses data obtained from differt sources or methods."

In [ ]:
# # List all datasets 
# existing_datasets <- openhexa$workspaces$workspace$list_datasets()

# # Check if a dataset with slug "dhis2-reporting-rate" exists (DATASET_SLUG)
# dataset_exists <- FALSE
# for (ds in existing_datasets) {
#   if (ds$slug == DATASET_SLUG) {
#     dataset_exists <- TRUE
#     dataset <- ds  # Store the existing dataset to be used downstream
#     break
#   }
# }

# # If it doesn't exist, create it 
# if (!dataset_exists) {
#   # Define name & description (modify as needed)
#   dataset_name <- DATASET_NAME
#   dataset_description <- DATASET_DESCRIPTION
  
#   # Create the dataset
#   dataset <- openhexa$workspaces$workspace$create_dataset(
#     name = dataset_name,
#     description = dataset_description
#   )
  
#   print(paste("Created new dataset:", dataset$slug))
# } else {
#   print(paste("Dataset already exists:", dataset$slug))
# }

### 4.2.1. Assign **version** to dataset - _if_ it does not have one yet
🚨 Note: brand-new datasets don't have a version yet! Version needs to be explicitly created! <br>
With this code a "**v0**" is created only if dataset doesn't have any (else it will keep its current version).

In [ ]:
# # Check if dataset has any versions
# if (is.null(dataset$latest_version)) {
#   print("Dataset has no versions. Creating initial version 'v0'...")
#   initial_version <- dataset$create_version("v0")
#   print(paste("Created version:", initial_version$name))
# } else {
#   print(paste("Dataset already has versions. Latest version:", dataset$latest_version$name))
# }

### 4.2.2. Define list of **files to be written** to the Dataset
Here we're writing _all_ the files that we previously exported as `.csv` and `.parquet` files to `/data/dhis2_incidence/`

In [ ]:
# EXPORTED_DATA_FOLDER = "dhis2_incidence"

In [ ]:
# # Define the directory path (with expanded ~)
# dir_path <- path.expand(file.path(DATA_PATH, EXPORTED_DATA_FOLDER))

# # List all files in the directory
# file_list <- list.files(dir_path, full.names = TRUE)

# # Check if there are any files
# if (length(file_list) == 0) {
#   stop("No files found in directory: ", dir_path)
# }

# # Print files to be uploaded (for debugging)
# print("Files to be uploaded:")
# print(file_list)

### 4.2.3. Create **new version** of Dataset and **write all files** to it
At each execution of the notebook (hence at each pipeline execution):
* **creates** a new version of the Dataset
* **writes** all files found in `file_list` to this (latest) Dataset version

In [ ]:
# # 1. Get latest dataset version name
# latest_version_name <- dataset$latest_version$name
# latest_version_nr <- as.numeric(gsub("v", "", latest_version_name))

# # 2. Create a new version (vX+1)
# new_version <- dataset$create_version(paste0("v", latest_version_nr + 1))

# # 3. Add all files to the new version
# for (file_path in file_list) {
#   # Extract just the filename (without full path)
#   filename <- basename(file_path)
  
#   # Upload the file
#   new_version$add_file(file_path, filename = filename)
  
#   # Print confirmation
#   print(paste("Uploaded:", filename))
# }

# print(paste("Successfully uploaded", length(file_list), "files to version", new_version$name))